In [1]:
import os
import shutil
import cv2 as cv
import json
import random
from shutil import copyfile
import concurrent.futures
from PIL import Image

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
root_path = "drive/MyDrive"

In [4]:
path = os.path.join(root_path,"images/publaynet/train_3.json")
with open(path,"rb") as file:
  train_dict = json.load(file)

In [5]:
index = 0
for image in train_dict.values():

   anno_name = ["figure_"+ str(index + (idx + 1)) if category == 1 else "not_figure_"+ str(index + (idx + 1)) \
   for idx, category in enumerate(image["annotations_category"])]
   image["annotations_name"] = anno_name
   index+= len(anno_name)

In [6]:
!ls

drive  sample_data


In [7]:
images_root_path = os.path.join(root_path,"images/publaynet/train")

In [8]:
os.makedirs("data")
os.makedirs("data/figure")
os.makedirs("data/not_figure")
figure_folder = "data/figure"
not_figure_folder = "data/not_figure"

In [14]:
print(train_dict.values())

Output hidden; open in https://colab.research.google.com to view.

In [15]:
def image_croping(image_dict):

  image_path = os.path.join(images_root_path,image_dict[0])
  image = Image.open(image_path)
  
  for idx, bbox  in enumerate(image_dict[1]["annotations_bbox"]):

    try:
      box_image = image.crop((bbox[0], bbox[1], bbox[0]+bbox[2], bbox[1]+bbox[3]))
      if image_dict[1]["annotations_category"][idx]:
        box_image = box_image.save(os.path.join(figure_folder,image_dict[1]["annotations_name"][idx]+".jpg"))
        
      else:
        box_image = box_image.save(os.path.join(not_figure_folder,image_dict[1]["annotations_name"][idx]+".jpg"))
    except:pass

    


In [16]:
import time

train_list = list(train_dict.items())

for image in os.listdir(figure_folder):
  os.remove(os.path.join(figure_folder,image))

for image in os.listdir(not_figure_folder):
  os.remove(os.path.join(not_figure_folder,image))

t1 = time.perf_counter()
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(image_croping, train_list)

"""for image in train_list:
  image_croping(image)"""

t2 = time.perf_counter()


print(f'Finished in {round(t2-t1,2)} secondes')

Finished in 687.54 secondes


In [17]:
print(len(os.listdir(figure_folder)))
print(len(os.listdir(not_figure_folder)))

11999
12000


In [18]:
!zip -r "/content/pdf_images.zip" "/content/data"

Streaming output truncated to the last 5000 lines.
  adding: content/data/figure/figure_18688.jpg (deflated 16%)
  adding: content/data/figure/figure_17589.jpg (deflated 14%)
  adding: content/data/figure/figure_17951.jpg (deflated 19%)
  adding: content/data/figure/figure_18023.jpg (deflated 26%)
  adding: content/data/figure/figure_1919.jpg (deflated 22%)
  adding: content/data/figure/figure_20194.jpg (deflated 11%)
  adding: content/data/figure/figure_7910.jpg (deflated 12%)
  adding: content/data/figure/figure_15650.jpg (deflated 30%)
  adding: content/data/figure/figure_19205.jpg (deflated 38%)
  adding: content/data/figure/figure_2905.jpg (deflated 17%)
  adding: content/data/figure/figure_12714.jpg (deflated 1%)
  adding: content/data/figure/figure_20791.jpg (deflated 31%)
  adding: content/data/figure/figure_2031.jpg (deflated 4%)
  adding: content/data/figure/figure_22571.jpg (deflated 6%)
  adding: content/data/figure/figure_16714.jpg (deflated 21%)
  adding: content/data/fig

### Create a dictionary having as keys images id and as values two lists one for the bbox and the other for their categories

to avoid opening images many times (the case where the image have more than one annotation) or pass through all annotation for each images

In [ ]:
def create_train_test_dirs(root_path):
 
  train_dir = os.path.join(root_path,"training")
  test_dir = os.path.join(root_path,"testing")
  train_dir_figure = os.path.join(train_dir,"figure")
  train_dir_not_figure = os.path.join(train_dir,"not_figure")
  test_dir_figure = os.path.join(test_dir,"figure")
  test_dir_not_figure = os.path.join(test_dir,"not_figure")
  os.makedirs(root_path)
  os.makedirs(train_dir)
  os.makedirs(test_dir)
  os.makedirs(train_dir_figure)
  os.makedirs(train_dir_not_figure)
  os.makedirs(test_dir_figure)
  os.makedirs(test_dir_not_figure)
  

In [ ]:
create_train_test_dirs("dataset")

In [ ]:
def split_training_test(source, training, testing, split_size):

  images = os.listdir(source)

  for image in images:
    if os.path.getsize(os.path.join(source,image)) < 0:
       print(f"image {image} is zero length")
       #os.remove(os.path.join(source,image))
  images = os.listdir(source)
  images = random.sample(len(images), images)
  train_size = int(split_size*len(images))

  for image in images[:train_size]:
    image_source = os.path.join(source,image)
    image_destination = os.path.join(training,image)
    copyfile(image_source, image_destination)

  for image in images[train_size:]:
    image_source = os.path.join(source,image)
    image_destination = os.path.join(testing,image)
    copyfile(image_source, image_destination)